In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string

2023-11-26 22:51:15.783837: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-26 22:51:16.106094: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 22:51:16.106167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 22:51:16.167358: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-26 22:51:16.288716: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-26 22:51:16.290380: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
df=pd.read_csv('one_output_dataset.csv')
df

,speech,emotion
0,From the moment that the French defenses at Se...,anger
1,We observe today not a victory of party but a ...,joy
2,"Your Majesties, Your Highnesses, Distinguished...",trust
3,I am honored to be with you today at your comm...,anticipation
4,"Honorable UN Secretary General Mr Ban Ki-moon,...",sadness
5,It is with a profound sense of humility that I...,sadness
6,My message is that we'll be watching you.\r\n\...,fear
7,"Hello everybody. You know, Michelle and I have...",joy
8,"Your Majesties, Your Royal Highness, Excellenc...",anticipation
9,"Five score years ago, a great American, in who...",trust


In [3]:
dict={'anger':0.0, 'joy':1.0, 'trust':2.0, 'anticipation':3.0, 'sadness':4.0, 'fear':5.0}
df=df.replace(dict)
df['emotion'] = df['emotion'].apply(pd.to_numeric)
df

,speech,emotion
0,From the moment that the French defenses at Se...,0.0
1,We observe today not a victory of party but a ...,1.0
2,"Your Majesties, Your Highnesses, Distinguished...",2.0
3,I am honored to be with you today at your comm...,3.0
4,"Honorable UN Secretary General Mr Ban Ki-moon,...",4.0
5,It is with a profound sense of humility that I...,4.0
6,My message is that we'll be watching you.\r\n\...,5.0
7,"Hello everybody. You know, Michelle and I have...",1.0
8,"Your Majesties, Your Royal Highness, Excellenc...",3.0
9,"Five score years ago, a great American, in who...",2.0


# New Section

In [4]:
#preprocess transcript

# download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# initialize a PorterStemmer
stemmer = PorterStemmer()

def preprocess_text(text):
    # lowercase the text
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # tokenize the text
    words = word_tokenize(text)
    # remove stopwords and stem the words
    words = [stemmer.stem(word) for word in words if word not in stopwords.words('english')]
    # join the words back into a string
    text = ' '.join(words)

    return text

df['speech'] = df['speech'].apply(preprocess_text)

[nltk_data] Error loading punkt: <urlopen error [Errno 104] Connection
[nltk_data]     reset by peer>
[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>


In [5]:
!pip install sentencepiece

In [6]:
tokenizer=AlbertTokenizer.from_pretrained('albert-large-v2')

model=TFAlbertForSequenceClassification.from_pretrained('albert-large-v2',num_labels=6)

2023-11-26 22:51:45.189296: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-26 22:51:45.189746: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifie

In [7]:
df

,speech,emotion
0,moment french defens sedan meus broken end sec...,0.0
1,observ today victori parti celebr freedomsymbo...,1.0
2,majesti high distinguish guest comrad friend t...,2.0
3,honor today commenc one finest univers world n...,3.0
4,honor un secretari gener mr ban kimoon respect...,4.0
5,profound sens humil accept honor chosen bestow...,4.0
6,messag well watch wrong shouldnt back school s...,5.0
7,hello everybodi know michel realli milk goodby...,1.0
8,majesti royal high excel distinguish member no...,3.0
9,five score year ago great american whose symbo...,2.0


In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [9]:
#split in train and test in text and labels
train_texts = train['speech'].tolist()
train_labels = train['emotion'].tolist()

test_texts = test['speech'].tolist()
test_labels = test['emotion'].tolist()

In [10]:
#tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

In [11]:
#convert features and labels to tensors for both train and test
train_features = {key: tf.convert_to_tensor(val) for key, val in train_encodings.items()}
train_labels = tf.convert_to_tensor(train_labels)

test_features = {key: tf.convert_to_tensor(val) for key, val in test_encodings.items()}
test_labels = tf.convert_to_tensor(test_labels)

In [12]:
#prepare the training and testing dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
train_dataset = train_dataset.shuffle(10000).batch(1)

test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels))
test_dataset = test_dataset.batch(1)

In [13]:
#define model metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

In [14]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [15]:
model.summary()

Model: "tf_albert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 albert (TFAlbertMainLayer)  multiple                  17683968  
                                                                 
 dropout_4 (Dropout)         multiple                  0         
                                                                 
 classifier (Dense)          multiple                  6150      
                                                                 
Total params: 17690118 (67.48 MB)
Trainable params: 17690118 (67.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
train_labels

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([0., 4., 2., 1., 4., 3., 1., 3.], dtype=float32)>

In [17]:
#fit model to train dataset
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

Epoch 1/3
8/8 [==============================] - 72s 7s/step - loss: 1.7633 - accuracy: 0.2500 - val_loss: 1.7402 - val_accuracy: 0.0000e+00
Epoch 2/3
8/8 [==============================] - 50s 6s/step - loss: 1.7886 - accuracy: 0.2500 - val_loss: 2.0379 - val_accuracy: 0.0000e+00
Epoch 3/3
8/8 [==============================] - 52s 7s/step - loss: 1.9025 - accuracy: 0.2500 - val_loss: 1.7425 - val_accuracy: 0.0000e+00


In [18]:
model.save('first_model.keras')

/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [25]:
df['speech'][1]

'observ today victori parti celebr freedomsymbol end well beginningsignifi renew well chang sworn almighti god solemn oath forbear prescrib nearli centuri threequart ago world differ man hold mortal hand power abolish form human poverti form human life yet revolutionari belief forebear fought still issu around globeth belief right man come generos state hand god dare forget today heir first revolut let word go forth time place friend foe alik torch pass new gener americansborn centuri temper war disciplin hard bitter peac proud ancient heritageand unwil wit permit slow undo human right nation alway commit commit today home around world let everi nation know whether wish us well ill shall pay price bear burden meet hardship support friend oppos foe assur surviv success liberti much pledgeand old alli whose cultur spiritu origin share pledg loyalti faith friend unit littl host cooper ventur divid littl dofor dare meet power challeng odd split asund new state welcom rank free pledg word o

In [26]:
new_feature = tokenizer(df['speech'][1], truncation=True, padding=True, return_tensors='tf')

In [27]:
predictions = model(new_feature)

In [28]:
predictions

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[-0.16195449, -0.0142893 ,  0.5625099 , -0.50989616,  0.69086015,
        -0.14531057]], dtype=float32)>, hidden_states=None, attentions=None)

In [29]:
probabilities = tf.nn.softmax(predictions.logits, axis=-1)

In [30]:
probabilities

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[0.1205994 , 0.13978976, 0.24887201, 0.08516007, 0.28295532,
        0.12262345]], dtype=float32)>